In [ ]:
import atfml

In [ ]:
def build_dataset():
    pass

In [ ]:
# for custom bundle
from atfml.core import UnitBundle
from atfml.utils import inits

class BundleName(UnitBundle):
    def __init__(self, bk, name, n_hid):
        arg_dict = {
            'name': name,
            'weight_template': {
                'w': {'shape': (n_hid, n_hid), 
                      'init_method': inits.identity_repeat_init 
                     },
            },
            'data_template': {
                'X': ('batch_size', n_hid),
            }
        }
        super().__init__(bk, **arg_dict)
        
    def _func(self, theta, data, const):
        np = None
        bk = self.bk
        return bk.dot(data.X, theta.w)

In [ ]:
# for custom model loss

from atfml.core import AutoGradBackend, TheanoBackend, UnitBundle
import atfml.utils as utils
from atfml.utils import inits, behaviours as bhvs
import atfml.bundles

bk = AutoGradBackend()
# bk = TheanoBackend()

class Model(bk.ModelLoss):
    def __init__(self, *, n_input_dim, n_hidden_dim):      
        self.n_input_dim = n_input_dim
        self.n_hidden_dim = n_hidden_dim
        
        arg_dict = {
            'default_init_method': inits.gaussian_init_with(mu=0, std=1),
            'weight_template': {
                'A': {'shape':(self.vocab_size, self.n_embedding_dim) },
            },
            'weight_bundles': {
                'sub_bundle': {
                    'class': BundleName, 
                    'args': {'n_hid':n_hidden_dim, },
                },
            },
            'data_template': {
                'X': {'shape':('batch_size', self.seq_len), 'dtype': 'int64' },
            },
            'optimization_method': {'name': 'adam', 'learning_rate': 0.005, 'clip': 100},
            'behaviours': {
                'loss_bhv': bhvs.LossLogBehaviour(),
            }
        }
        super().__init__(**arg_dict)
    
    @bk.export
    def predict(self, theta, data, const):
        np = None # for safety
        pass
        return data.X
    
    def loss(self, theta, data, const):
        np = None # for safety
        pred = self.raw.predict(theta, data, const)
        loss = None
        return loss
    
    @bk.export
    def score(self, theta, data, const):
        np = None # for safety
        # ...
        score = None
        return score
    
    def step_callback(self, loss_val, theta, data, const, info):
        bk = None
        if info['n_iter'] % 50 == 0:
            score_val = self.compiled.score(theta, data, const)
            print('%d %10.4g %10.4g' % (info['n_iter'], loss_val, score_val))

def test():
    data = build_dataset()
    model = Model(n_input_dim=10, n_hidden_dim=10)
    best_theta = model.fit(data, n_max_steps=2000)
    plt.plot(model.behaviours.loss_bhv.log)
    plt.show()
    
test()